# Assigment 1
### 02467 Computational Social Science Group 6

# Part 1: Web-scraping
### _Exercise: Web-scraping the list of participants to the International Conference in Computational Social Science_

In [ ]:
#code for part 1

### _How many unique researchers do you get?_
#### We got 1484 unique researchers for our answer.

### _Explain the process you followed to web-scrape the page. Which choices did you make to accurately retreive as many names as possible? Which strategies did you use to assess the quality of your final list? Explain your reasoning and your choices (answer in max 150 words)._
#### We first cleaned the names by removing any special character and/or unnecessary white spaces and commas, and excluded names that were too short or contained numbers. We then removed the word "Chair" from the session chairs whose names were saved as "Chair: (name)". Lastly, we added these cleaned names into a list. Using the list, we retrieved the information needed using BeautifulSoup. We then sorted the list by the names to get our final answer.

### Part 3: Gathering Research Articles using the OpenAlex API

In [ ]:
import requests
import pandas as pd
from joblib import Parallel, delayed

df = pd.read_csv('file02.csv')
base_url = "https://api.openalex.org/"
papers = []
abstracts = []

socialscience = {
    "Sociology": "https://openalex.org/C144024400",
    "Psychology": "https://openalex.org/C15744967",
    "Economics": "https://openalex.org/C162324750",
    "Political Science": "https://openalex.org/C17744445"
}

quantitative = {
    "Mathematics": "https://openalex.org/C33923547",
    "Physics": "https://openalex.org/C121332964",
    "Computer Science": "https://openalex.org/C41008148"
}

def getwork(works_api_url, per_page=200):
    works = []
    cursor = "*"  # initial cursor
    while True:
        url = f"{works_api_url}&per-page={per_page}&cursor={cursor}"
        response = requests.get(url)
        if response.status_code != 200: break
        
        data = response.json()
        works.extend(data.get('results', []))
        
        # next cursor for pagination
        next_cursor = data.get('meta', {}).get('next_cursor')
        if not next_cursor: break
        cursor = next_cursor  
    return works

def filtering(works):
    filtered = []
    for work in works:
        if work.get('cited_by_count', 0) <= 10: continue  #more than 10 citations
        if len(work.get('authorships', [])) >= 10: continue #fewer than 10 authors
        
        concept_ids = [concept.get('id') for concept in work.get('concepts', [])]
        is_SS = any(concept in socialscience.values() for concept in concept_ids)
        is_quant = any(concept in quantitative.values() for concept in concept_ids)

        if is_SS and is_quant: #works relevant to Computational Social Science  AND intersecting with a quantitative discipline
            filtered.append(work)
    return filtered

def filtering2(id, worksurl, count): # Only if the author has between 5 and 5000 works
    if 5 <= count <= 5000:
        works = getwork(worksurl)
        filtered_works = filtering(works)
        return filtered_works
    return []

def extraction(work):
    return {
        'id': work.get('id'),
        'publication_year': work.get('publication_year'),
        'cited_by_count': work.get('cited_by_count'),
        'author_ids': [author.get('author', {}).get('id') for author in work.get('authorships', [])],
        'title': work.get('title'),
        'abstract_inverted_index': work.get('abstract_inverted_index'),
        'referenced_works': work.get('referenced_works', []), 
        'cited_by_api_url': work.get('cited_by_api_url'),
        'related_works': work.get('related_works', []) 
    }

# parallelize fetching and filtering works using joblib
allfiltered = Parallel(n_jobs=-1)(
    delayed(filtering2)(row['OpenAlex ID'], row['Works API URL'], row['Works Count'])
    for _, row in df.iterrows()
)

for all in allfiltered:
    for work in all:
        details = extraction(work)
    
        papers.append({
            'id': details['id'],
            'publication_year': details['publication_year'],
            'cited_by_count': details['cited_by_count'],
            'author_ids': details['author_ids'],
            'referenced_works': details['referenced_works'],
            'cited_by_api_url': details['cited_by_api_url'],
            'related_works': details['related_works']
        })
        
        abstracts.append({
            'id': details['id'],
            'title': details['title'],
            'abstract_inverted_index': details['abstract_inverted_index']
        })

papers_df = pd.DataFrame(papers)
abstracts_df = pd.DataFrame(abstracts)

papers_df.to_csv('papers.csv', index=False)
abstracts_df.to_csv('abstracts.csv', index=False)
print("Data saved.")

### Data Overview and Reflection questions:

#### Dataset summary. How many works are listed in your IC2S2 papers dataframe? How many unique researchers have co-authored these works?

#### Efficiency in code. Describe the strategies you implemented to make your code more efficient. How did your approach affect your code's execution time? (answer in max 150 words)

#### Filtering Criteria and Dataset Relevance Reflect on the rationale behind setting specific thresholds for the total number of works by an author, the citation count, the number of authors per work, and the relevance of works to specific fields. How do these filtering criteria contribute to the relevance of the dataset you compiled? Do you believe any aspects of Computational Social Science research might be underrepresented or overrepresented as a result of these choices? (answer in max 150 words)